# Sparse Autoencoder Training Demo

## Setup

In [5]:
# Autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from sparse_autoencoder import TensorActivationStore, SparseAutoencoder, pipeline
from sparse_autoencoder.source_data.pile_uncopyrighted import PileUncopyrightedDataset
from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device
from transformers import PreTrainedTokenizerBase
import torch

In [7]:
device = get_device()

### Source Model

In [9]:
src_model = HookedTransformer.from_pretrained("solu-1l", dtype="float32")
src_d_mlp: int = src_model.cfg.d_mlp  # type: ignore
src_d_mlp

### Source Dataset

In [ ]:
tokenizer: PreTrainedTokenizerBase = src_model.tokenizer  # type: ignore
source_data = PileUncopyrightedDataset(tokenizer=tokenizer)

### Activation Store

In [ ]:
max_items = 1_000_000
store = TensorActivationStore(max_items, src_d_mlp, device)

### Autoencoder

In [ ]:
autoencoder = SparseAutoencoder(src_d_mlp, src_d_mlp * 8, torch.zeros(src_d_mlp))
autoencoder

## Training

If you initialise [wandb](https://wandb.ai/site), the pipeline will automatically log all metrics to wandb.

In [ ]:
# wandb.init(project="sparse-autoencoder", dir=".cache/wandb")

In [ ]:
pipeline(
    src_model=src_model,
    src_model_activation_hook_point="blocks.0.mlp.hook_post",
    src_model_activation_layer=0,
    source_dataset=source_data,
    activation_store=store,
    num_activations_before_training=max_items,
    autoencoder=autoencoder,
    device=device,
)